In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


Losses and metrics are conceptually not the same thing: **losses are used by Gradient Descent to train a model**, so they must be differentiable (at least where they are evaluated) and their gradients should not be 0 everywhere. Plus, it’s okay if they are not easily interpretable by humans (e.g. cross-entropy). In contrast, metrics are used to evaluate a model, they must be more easily interpretable, and they can be nondifferentiable or have 0 gradients everywhere (e.g., accuracy).

That said, in most cases, defining a custom metric function is exactly the same as
defining a custom loss function. In fact, we could even use the Huber loss function we
created earlier in **22_implementing_custom_loss_and_saving_the_model.ipynb**, it would work just fine (and persistence would also work the same way, in this case only saving the name of the function, "huber_floss" ):
```
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])
```

In [2]:
precision = tf.keras.metrics.Precision()

In [3]:
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: id=67, shape=(), dtype=float32, numpy=0.8>

In [4]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: id=117, shape=(), dtype=float32, numpy=0.5>

In [5]:
precision.result()

<tf.Tensor: id=127, shape=(), dtype=float32, numpy=0.5>

In [7]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [9]:
precision.reset_states()

In [10]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

If you need to create such a streaming metric, you can just create a subclass of the
keras.metrics.Metric class. Here is a simple example that keeps track of the total
Huber loss and the number of instances seen so far. When asked for the result, it
returns the ratio, which is simply the mean Huber loss:

In [11]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [13]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

• The constructor uses the add_weight() method to create the variables needed to
keep track of the metric’s state over multiple batches, in this case the sum of all
Huber losses ( total ) and the number of instances seen so far ( count ). You could
just create variables manually if you preferred. Keras tracks any tf.Variable that
is set as an attribute (and more generally, any “trackable” object, such as layers or
models).

• The update_state() method is called when you use an instance of this class as a
function (as we did with the Precision object). It updates the variables given the
labels and predictions for one batch (and sample weights, but in this case we just
ignore them).

• The result() method computes and returns the final result, in this case just the
mean Huber metric over all instances. When you use the metric as a function, the
update_state() method gets called first, then the result() method is called,
and its output is returned.

• We also implement the get_config() method to ensure the threshold gets
saved along with the model.

• The default implementation of the reset_states() method just resets all 
variables to 0.0 (but you can override it if needed).

*When you define a metric using a simple function, Keras automatically calls it for
each batch, and it keeps track of the mean during each epoch, just like we did manually. 
So the only benefit of our HuberMetric class is that the threshold will be saved.
But of course, some metrics, like precision, cannot simply be averaged over batches:
in thoses cases, there’s no other option than to implement a streaming metric.*